In [3]:
# Gerekli kütüphaneler
import json
import csv
import random
import re
from pathlib import Path
from typing import List, Dict, Tuple, Optional

DATA_DIR = Path('/home/yusuf/teknofest')
CSV_PATH = DATA_DIR / 'deneme.csv'
JSON_PATH = DATA_DIR / 'muhtarlikdata.json'  # her satır bir JSON obje olabilir varsayımı

random.seed(42)

# ----------------------------------------------------------------------------------
# KONFİG: İstanbul ekleme stratejisi
# ----------------------------------------------------------------------------------
# istanbul_mode: 'always' -> her örnekte IL=İstanbul ekle
#                 'ratio'  -> belirli oranda ekle
#                 'never'  -> hiç ekleme (JSON/CSV içinde varsa yine eklenebilir)
istanbul_mode = 'ratio'
istanbul_ratio = 0.7  # biraz daha yüksek oran

# istanbul_position: 'start' | 'middle' | 'end' | 'mixed' -> metinde konum çeşitliliği
istanbul_position = 'mixed'

# Üretim kalite kısıtları
MIN_TOKENS = 5
REQUIRE_CORE = True  # en azından MAHALLE veya ILCE veya ROUTE içersin

# ----------------------------------------------------------------------------------
# Basit normalizasyonlar

def norm(s: Optional[str]) -> str:
    if s is None:
        return ''
    s = s.strip()
    # fazla boşlukları tek boşluğa indir
    s = re.sub(r'\s+', ' ', s)
    return s

# Türkçe özel: Büyük/küçük harf varyasyonları için basit fonk

def cap_first(token: str) -> str:
    return token[:1].upper() + token[1:]

# Kısaltma/tam yazım varyasyon sözlüğü
ABBR = {
    'MAHALLE': [('Mahallesi', 'Mah.'), ('Mahallesi', 'Mh.'), ('Mahallesi', 'Mahalle')],
    'CADDE': [('Caddesi', 'Cd.'), ('Caddesi', 'Cad.'), ('Caddesi', 'C.'), ('Caddesi', 'Cadde')],
    'SOKAK': [('Sokağı', 'Sk.'), ('Sokağı', 'Sok.'), ('Sokağı', 'Sokak')],
    'NO': [('No:', 'No'), ('No:', 'No.'), ('No:', 'Nr.'), ('No:', 'No :')],
}

# Basit tokenizasyon (noktalama ayır)
TOKEN_SPLIT = re.compile(r"([\wçğıöşüÇĞİÖŞÜ]+|\d+|:\d+|[:.,/()\-])", re.UNICODE)

def tokenize(text: str) -> List[str]:
    parts = TOKEN_SPLIT.findall(text)
    tokens = [p for p in parts if p and not p.isspace()]
    return tokens

# BIO etiketleyici yardımcıları

def label_span(tokens: List[str], start_idx: int, length: int, tag: str, labels: List[str]):
    if length <= 0:
        return
    labels[start_idx] = f'B-{tag}'
    for i in range(1, length):
        labels[start_idx + i] = f'I-{tag}'

# Basit adres şablonları (İstanbul'u farklı konumlarda ve POI ile destekler)
BASE_TEMPLATES = [
    # mahalle + cadde/sokak + no + ilçe
    ['MAHALLE', 'ROUTE', 'NO', 'ILCE'],
    # ilçe + mahalle + sokak + no
    ['ILCE', 'MAHALLE', 'ROUTE', 'NO'],
    # mahalle + rota + ilçe (il eksik)
    ['MAHALLE', 'ROUTE', 'ILCE'],
    # rota + mahalle (ilçe eksik)
    ['ROUTE', 'MAHALLE'],
    # mahalle + ilçe
    ['MAHALLE', 'ILCE'],
    # POI + mahalle + rota
    ['POI', 'MAHALLE', 'ROUTE'],
    # mahalle + POI + ilçe
    ['MAHALLE', 'POI', 'ILCE'],
]

# İstanbul konumlu varyantları türet
TEMPLATES: List[List[str]] = []
for base in BASE_TEMPLATES:
    # start
    TEMPLATES.append(['IL@START'] + base)
    # middle
    mid_idx = 1 if len(base) >= 2 else len(base)
    TEMPLATES.append(base[:mid_idx] + ['IL@MID'] + base[mid_idx:])
    # end
    TEMPLATES.append(base + ['IL@END'])

# Rota: cadde veya sokak

def pick_route(cadde: str, sokak: str) -> Tuple[str, str]:
    cadde = norm(cadde)
    sokak = norm(sokak)
    if cadde and sokak:
        # rastgele birini seç
        if random.random() < 0.6:
            return cadde, 'CADDE'
        else:
            return sokak, 'SOKAK'
    if cadde:
        return cadde, 'CADDE'
    if sokak:
        return sokak, 'SOKAK'
    return '', ''

# Kısaltma varyasyonu uygula

def vary_form(kind: str, core: str) -> List[str]:
    core = norm(core)
    out: List[str] = []
    if not core:
        return out
    if kind == 'MAHALLE':
        for full, abbr in ABBR['MAHALLE']:
            out.append(f"{core} {abbr}")
        out.append(f"{core} Mahallesi")
    elif kind == 'CADDE':
        for full, abbr in ABBR['CADDE']:
            out.append(f"{core} {abbr}")
        out.append(f"{core} Caddesi")
    elif kind == 'SOKAK':
        for full, abbr in ABBR['SOKAK']:
            out.append(f"{core} {abbr}")
        out.append(f"{core} Sokağı")
    elif kind == 'NO':
        num = core
        for full, abbr in ABBR['NO']:
            out.append(f"{abbr} {num}")
        out.append(f"No: {num}")
    elif kind == 'ILCE':
        # ilçe yalın veya 'İlçesi'
        out.append(core)
        out.append(f"{core} İlçesi")
    elif kind == 'POI':
        # POI için farklı ekler: Pazar, Pazar Yeri, Muhtarlığı vb.
        poi = core
        out.append(poi)
        out.append(f"{poi} Pazar")
        out.append(f"{poi} Pazarı")
        out.append(f"{poi} Pazar Yeri")
        out.append(f"{poi} Muhtarlığı")
    return list(dict.fromkeys(out))  # unique, preserve order

# Bir bileşeni BIO ile etiketle

def place_component(tokens: List[str], labels: List[str], phrase: str, tag: str):
    phrase_tokens = tokenize(phrase)
    start = len(tokens)
    tokens.extend(phrase_tokens)
    labels.extend(['O'] * len(phrase_tokens))
    label_span(tokens, start, len(phrase_tokens), tag, labels)

# Noktalama/ayırıcı ekleyen yardımcı

def add_sep(tokens: List[str], labels: List[str], sep: str = ','):
    if tokens and tokens[-1] not in [',', '/', '-']:
        tokens.append(sep)
        labels.append('O')

# JSON satırlarını oku (line-delimited json veya tek obje dizi)

def read_json_records(path: Path) -> List[Dict]:
    if not path.exists():
        return []
    txt = path.read_text(encoding='utf-8').strip()
    if not txt:
        return []
    recs: List[Dict] = []
    # line-delimited ise
    if '\n{' in txt and not txt.strip().startswith('['):
        for line in txt.splitlines():
            line = line.strip()
            if not line:
                continue
            try:
                recs.append(json.loads(line))
            except Exception:
                # tek JSON obje olabilir
                try:
                    obj = json.loads(txt)
                    if isinstance(obj, dict):
                        recs.append(obj)
                        break
                except Exception:
                    pass
    else:
        try:
            obj = json.loads(txt)
            if isinstance(obj, list):
                recs.extend(obj)
            elif isinstance(obj, dict):
                recs.append(obj)
        except Exception:
            pass
    return recs

# CSV'den örnek adres kırp

def iter_csv_rows(path: Path, limit: int = 50):
    if not path.exists():
        return
    with path.open('r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for i, row in enumerate(reader):
            yield row
            if i + 1 >= limit:
                break

# JSON kayıtlarından alanları çek

def extract_from_json(rec: Dict) -> Dict[str, str]:
    props = rec.get('properties') or rec
    ilce = norm(props.get('İlçe Adı') or props.get('ilce') or '')
    il = norm(props.get('İl Adı') or props.get('il') or 'İstanbul')  # varsayılan İstanbul
    mahalle = norm(props.get('Mahalle Adı') or props.get('mahalle') or '')
    adres = norm(props.get('Adres') or '')
    poi = norm(props.get('Muhtarlık Adı') or props.get('poi') or '')
    # Adresten basit No bul
    m = re.search(r'(?:No\s*[:.]?\s*)(\d+)', adres, flags=re.IGNORECASE)
    no_val = m.group(1) if m else ''
    # Sokak benzeri (Çıkmazı, Sokak, Cd, Sk)
    m2 = re.search(r'([\wçğıöşüÇĞİÖŞÜ]+(?:\s+(?:Cd\.?|Caddesi|Sokak|Sk\.?|Sokağı|Çıkmazı)))', adres, flags=re.IGNORECASE)
    rota_core = ''
    rota_kind = ''
    if m2:
        phrase = m2.group(1)
        # çekirdek isim + türü ayırmaya çalış
        if re.search(r'Cad|Cd\.|Caddesi', phrase, flags=re.IGNORECASE):
            rota_kind = 'CADDE'
            rota_core = re.sub(r'\s+(?:Cad(?:desi)?|Cd\.|Caddesi)$', '', phrase, flags=re.IGNORECASE)
        elif re.search(r'Sok|Sk\.|Sokağı|Çıkmazı', phrase, flags=re.IGNORECASE):
            rota_kind = 'SOKAK'
            rota_core = re.sub(r'\s+(?:Sok(?:ak)?|Sk\.|Sokağı|Çıkmazı)$', '', phrase, flags=re.IGNORECASE)
    return {
        'IL': il.title() if il else '',
        'ILCE': cap_first(ilce.lower()) if ilce else '',
        'MAHALLE': cap_first(mahalle.lower()) if mahalle else '',
        'NO': no_val,
        'ROUTE_CORE': cap_first(rota_core.lower()) if rota_core else '',
        'ROUTE_KIND': rota_kind,
        'POI': poi,
    }

# CSV satırından alanları çek

def extract_from_csv(row: Dict[str, str]) -> Dict[str, str]:
    il = 'İstanbul'  # veri ilçe bazlı, il genelde İstanbul
    ilce = norm(row.get('İlçe') or '')
    mahalle = norm(row.get('Mahalle') or '')
    cadde = norm(row.get('Cadde') or '')
    sokak = norm(row.get('Sokak') or '')
    poi = norm(row.get('Pazar Adı') or '')
    route_core, route_kind = pick_route(cadde, sokak)
    # route_core son eki sil (cd/sk vs)
    route_core = re.sub(r'\s+(?:cd\.?|cad\.?|caddesi|sokak|sk\.?|sokağı)$', '', route_core, flags=re.IGNORECASE)
    return {
        'IL': il,
        'ILCE': cap_first(ilce.lower()) if ilce else '',
        'MAHALLE': cap_first(re.sub(r'\s+mh\.?$', '', mahalle, flags=re.IGNORECASE).lower()) if mahalle else '',
        'NO': '',
        'ROUTE_CORE': cap_first(route_core.lower()) if route_core else '',
        'ROUTE_KIND': route_kind,
        'POI': poi,
    }

# İstanbul'u eklemeyi gerektiren karar

def should_add_istanbul(explicit_il: str) -> bool:
    if istanbul_mode == 'always':
        return True
    if istanbul_mode == 'never':
        return False
    # ratio
    if explicit_il and explicit_il.lower() in ['istanbul', 'i̇stanbul']:
        return True  # zaten var
    return random.random() < istanbul_ratio

# Uygun şablonları mevcut bileşenlere göre filtrele

def filter_templates_by_availability(templates: List[List[str]], components: Dict[str, str]) -> List[List[str]]:
    avail = {
        'MAHALLE': bool(components.get('MAHALLE')),
        'ROUTE': bool(components.get('ROUTE_CORE')),
        'NO': True,  # no üretilebilir
        'ILCE': bool(components.get('ILCE')),
        'POI': bool(components.get('POI')),
    }
    filtered = []
    for tpl in templates:
        ok = True
        for part in tpl:
            if part in avail and not avail[part]:
                ok = False
                break
        if ok:
            filtered.append(tpl)
    return filtered or templates  # hiç kalmazsa orijinal seti kullan

# Bir örnek üret

def build_example(components: Dict[str, str]) -> Optional[Dict]:
    il = components.get('IL', '')
    ilce = components.get('ILCE', '')
    mahalle = components.get('MAHALLE', '')
    route_core = components.get('ROUTE_CORE', '')
    route_kind = components.get('ROUTE_KIND', '')
    no_val = components.get('NO', '')
    poi = components.get('POI', '')

    # yoksa başarısız olma: en az bir parça şart
    if not any([ilce, mahalle, route_core, no_val, poi]):
        return None

    # İstanbul'u ekleme kararı ve pozisyonu
    add_il = should_add_istanbul(il)
    pos_choice = istanbul_position
    if istanbul_position == 'mixed':
        pos_choice = random.choice(['start', 'middle', 'end'])

    # mevcutlara uygun şablon seç
    candidates = filter_templates_by_availability(TEMPLATES, components)
    template = random.choice(candidates)

    tokens: List[str] = []
    labels: List[str] = []

    def maybe_sep():
        if tokens and random.random() < 0.35:
            add_sep(tokens, labels, ',')

    def place_il():
        place_component(tokens, labels, 'İstanbul', 'IL')

    for part in template:
        if part == 'IL@START':
            if add_il and pos_choice == 'start':
                place_il()
                continue
            continue
        if part == 'IL@MID':
            if add_il and pos_choice == 'middle':
                maybe_sep()
                place_il()
                continue
            continue
        if part == 'IL@END':
            # end ekleme loop sonunda yapılacak
            continue
        if part == 'MAHALLE' and mahalle:
            choices = vary_form('MAHALLE', mahalle)
            if not choices:
                continue
            phrase = random.choice(choices)
            maybe_sep()
            place_component(tokens, labels, phrase, 'MAHALLE')
        elif part == 'ROUTE' and route_core:
            kind = route_kind or ('CADDE' if random.random() < 0.5 else 'SOKAK')
            choices = vary_form(kind, route_core)
            if not choices:
                continue
            phrase = random.choice(choices)
            maybe_sep()
            place_component(tokens, labels, phrase, kind)
        elif part == 'NO':
            # yalnız başına olmasın; en azından rota/mahalle varsa koy
            if (route_core or mahalle or ilce) and (no_val or random.random() < 0.5):
                num = no_val or str(random.randint(1, 299))
                choices = vary_form('NO', num)
                if choices:
                    phrase = random.choice(choices)
                    maybe_sep()
                    place_component(tokens, labels, phrase, 'NO')
        elif part == 'ILCE' and ilce and (random.random() < 0.95):
            maybe_sep()
            # ilçe için varyantlar
            choices = vary_form('ILCE', ilce)
            phrase = random.choice(choices)
            place_component(tokens, labels, phrase, 'ILCE')
        elif part == 'POI' and poi and (random.random() < 0.7):
            maybe_sep()
            choices = vary_form('POI', poi)
            phrase = random.choice(choices)
            place_component(tokens, labels, phrase, 'POI')
        # eksik yazımı simüle etmek için boş geçilebilir

    # End konumunda İstanbul eklenecekse ve başka bileşen varsa
    if add_il and pos_choice == 'end' and tokens:
        maybe_sep()
        place_il()

    # kalite kontrolleri
    if not tokens:
        return None
    # En az bir çekirdek bileşen içeriyor mu?
    present_tags = {lab.split('-', 1)[1] for lab in labels if lab != 'O'}
    has_core = any(tag in present_tags for tag in ['MAHALLE', 'ROUTE', 'ILCE'])
    if REQUIRE_CORE and not has_core:
        return None
    # Sadece IL veya (IL+NO) olmasın
    if present_tags.issubset({'IL'}):
        return None
    if present_tags.issubset({'IL', 'NO'}):
        return None
    # token sayısı
    if len(tokens) < MIN_TOKENS:
        return None

    # metin üret
    text = ' '.join(tokens)

    return {
        'text': text,
        'tokens': tokens,
        'labels': labels,
    }

print('Yardımcılar yüklendi. İstanbul ekleme modu:', istanbul_mode)

Yardımcılar yüklendi. İstanbul ekleme modu: ratio


# Adres NER Veri Üretimi

Bu not defteri, yarı-yapısal adres verilerinden (CSV/JSON) BERT tabanlı NER eğitimi için BIO etiketli adres cümleleri üretir. Türkçe adres bileşenleri için şu etiketler kullanılır: IL, ILCE, MAHALLE, CADDE, SOKAK, NO, POI.


In [4]:
# Veri oku ve örnekler üret
examples = []

# JSON kayıtları
for rec in read_json_records(JSON_PATH):
    comp = extract_from_json(rec)
    # her kayıt için birkaç varyasyon isteyelim
    for _ in range(10):
        ex = build_example(comp)
        if ex:
            examples.append(ex)

# CSV satırları (ilk 200'e kadar)
for row in iter_csv_rows(CSV_PATH, limit=200):
    comp = extract_from_csv(row)
    for _ in range(8):
        ex = build_example(comp)
        if ex:
            examples.append(ex)

# Karıştır ve kalite açısından benzersiz metinleri koru
random.shuffle(examples)
unique = {}
for ex in examples:
    if ex['text'] and ex['text'] not in unique:
        unique[ex['text']] = ex
examples = list(unique.values())

print(f"Toplam örnek (filtrelenmiş): {len(examples)}")

# Örneklerden ilk 10'unu istenen formatta yazdır
for i, ex in enumerate(examples[:10], 1):
    item = {
        'text': ex['text'],
        'tokens': ex['tokens'],
        'labels': ex['labels'],
    }
    print(json.dumps(item, ensure_ascii=False))

# JSONL olarak kaydet
out_path = DATA_DIR / 'ner_adres_samples.jsonl'
with out_path.open('w', encoding='utf-8') as f:
    for ex in examples:
        f.write(json.dumps({
            'text': ex['text'],
            'tokens': ex['tokens'],
            'labels': ex['labels'],
        }, ensure_ascii=False) + "\n")
print('Kaydedildi:', str(out_path))

Toplam örnek (filtrelenmiş): 1556
{"text": "Alparslan türkeş C . Orhangazi Mahalle", "tokens": ["Alparslan", "türkeş", "C", ".", "Orhangazi", "Mahalle"], "labels": ["B-CADDE", "I-CADDE", "I-CADDE", "I-CADDE", "B-MAHALLE", "I-MAHALLE"]}
{"text": "Hicret Sk . , Mehmet akif ersoy Mh .", "tokens": ["Hicret", "Sk", ".", ",", "Mehmet", "akif", "ersoy", "Mh", "."], "labels": ["B-SOKAK", "I-SOKAK", "I-SOKAK", "O", "B-MAHALLE", "I-MAHALLE", "I-MAHALLE", "I-MAHALLE", "I-MAHALLE"]}
{"text": "Ömerli Ömerli Mahallesi Yamaç ve süngü Sokak İstanbul", "tokens": ["Ömerli", "Ömerli", "Mahallesi", "Yamaç", "ve", "süngü", "Sokak", "İstanbul"], "labels": ["B-POI", "B-MAHALLE", "I-MAHALLE", "B-SOKAK", "I-SOKAK", "I-SOKAK", "I-SOKAK", "B-IL"]}
{"text": "Esatpaşa Mh . Sorgun Sokağı", "tokens": ["Esatpaşa", "Mh", ".", "Sorgun", "Sokağı"], "labels": ["B-MAHALLE", "I-MAHALLE", "I-MAHALLE", "B-SOKAK", "I-SOKAK"]}
{"text": "Acem kızı Sok . Güzelyalı Mahalle", "tokens": ["Acem", "kızı", "Sok", ".", "Güzelyalı", "Ma